In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
import regex as re
from collections import Counter

In [2]:
df_master = pd.read_excel(r"data/SuperRare_master.xlsx")
df_bids = pd.read_csv(r"C:\Users\culm-erbach\Documents\VW NLP\requirements\data\bids.csv")

In [3]:
df_master

,Unnamed: 0,timestamp_x,tokenId,buyer,seller,eth,rate,usd,contract_x,transactionId_x,namedescriptiontags,description_category,image,media,type,size,dimensions,creator,owner,timestamp_y,contract_y,transactionId_y,tokenId_count_bids,usd_mean_bids,tokenId.1,timestamp_x.1,Covid_dummy,keyword_social_media,keyword_art_movement,keyword_gaming,keyword_dark,keyword_crypto,keyword_AI,keyword_nature,keyowrd_other,2018_Q2,2018_Q3,2018_Q4,2019_Q1,2019_Q2,2019_Q3,2019_Q4,2020_Q1,2020_Q2,2020_Q3,2020_Q4,2021_Q1,dollar
0,0,2018-04-05T23:50:12Z,1,0xbc74c3adc2aa6a85bda3eca5b0e235ca08532772,0x860c4604fe1125ea43f81e613e7afb2aa49546aa,0.46,381.36,1.754256e+02,0x41a322b28d0ff354040e2cbc676f0320d8c8850d,0xf1097e3617632e43b7c0a46ffeb4d741d0a67b25fb06...,"AI Generated Nude Portrait #1 Robbie Barrat, A...",6.0,https://ipfs.pixura.io/ipfs/QmX4ECCmA4HZSPxzsg...,NaN,NaN,NaN,NaN,0x860c4604fe1125ea43f81e613e7afb2aa49546aa,0xd0c0650cd08acd4e9553c48c60c94be04fecce43,2018-04-05T23:20:48Z,0x41a322b28d0ff354040e2cbc676f0320d8c8850d,0xf3e68d3a53b1bb3a2cdb4aa3a6c871626e6dcf7b8df1...,13,1.020731e+04,1,2018-04-05T23:50:12Z,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1.754256e+02
1,1,2020-01-18T16:16:42Z,1,0x54d7f921785ebe46010d83c73712e80dfaff1e81,0xbc74c3adc2aa6a85bda3eca5b0e235ca08532772,75.00,174.00,1.305000e+04,0x41a322b28d0ff354040e2cbc676f0320d8c8850d,0xf8d3b8be83601d0351c72d2093738a4a25c70b49503b...,"AI Generated Nude Portrait #1 Robbie Barrat, A...",6.0,https://ipfs.pixura.io/ipfs/QmX4ECCmA4HZSPxzsg...,NaN,NaN,NaN,NaN,0x860c4604fe1125ea43f81e613e7afb2aa49546aa,0xd0c0650cd08acd4e9553c48c60c94be04fecce43,2018-04-05T23:20:48Z,0x41a322b28d0ff354040e2cbc676f0320d8c8850d,0xf3e68d3a53b1bb3a2cdb4aa3a6c871626e6dcf7b8df1...,13,1.020731e+04,1,2020-01-18T16:16:42Z,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1.305000e+04
2,2,2021-01-05T00:47:24Z,1,0xd0c0650cd08acd4e9553c48c60c94be04fecce43,0x54d7f921785ebe46010d83c73712e80dfaff1e81,100.00,1103.19,1.103190e+05,0x41a322b28d0ff354040e2cbc676f0320d8c8850d,0xc917fe7d09a750c09fd8f467d60e5adac4bbd3a5e5ea...,"AI Generated Nude Portrait #1 Robbie Barrat, A...",6.0,https://ipfs.pixura.io/ipfs/QmX4ECCmA4HZSPxzsg...,NaN,NaN,NaN,NaN,0x860c4604fe1125ea43f81e613e7afb2aa49546aa,0xd0c0650cd08acd4e9553c48c60c94be04fecce43,2018-04-05T23:20:48Z,0x41a322b28d0ff354040e2cbc676f0320d8c8850d,0xf3e68d3a53b1bb3a2cdb4aa3a6c871626e6dcf7b8df1...,13,1.020731e+04,1,2021-01-05T00:47:24Z,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1.103190e+05
3,3,2018-04-05T23:59:06Z,2,0xbc74c3adc2aa6a85bda3eca5b0e235ca08532772,0x860c4604fe1125ea43f81e613e7afb2aa49546aa,0.46,381.36,1.754256e+02,0x41a322b28d0ff354040e2cbc676f0320d8c8850d,0x9e9befc01fe11425ae8a2b903c8f8e1ec9777157d3e9...,"AI Generated Nude Portrait #2 Robbie Barrat, A...",6.0,https://ipfs.pixura.io/ipfs/QmRe3WvttmMR7mELga...,NaN,NaN,NaN,NaN,0x860c4604fe1125ea43f81e613e7afb2aa49546aa,0x6853a596d6d7264d3622546da3b891b6fe17eb82,2018-04-05T23:49:27Z,0x41a322b28d0ff354040e2cbc676f0320d8c8850d,0x8fb08cb45e1a0032dccd0951812dba7a8ebe5b255bdd...,8,1.455821e+03,2,2018-04-05T23:59:06Z,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1.754256e+02
4,4,2018-09-05T21:06:04Z,2,0x6853a596d6d7264d3622546da3b891b6fe17eb82,0xbc74c3adc2aa6a85bda3eca5b0e235ca08532772,1.85,228.27,4.222995e+02,0x41a322b28d0ff354040e2cbc676f0320d8c8850d,0x6425dbd16550508c7c0ca320a1e6d7cb2ba2d3b2aea5...,"AI Generated Nude Portrait #2 Robbie Barrat, A...",6.0,https://ipfs.pixura.io/ipfs/QmRe3WvttmMR7mELga...,NaN,NaN,NaN,NaN,0x860c4604fe1125ea43f81e613e7afb2aa49546aa,0x6853a596d6d7264d3622546da3b891b6fe17eb82,2018-04-05T23:49:27Z,0x41a322b28d0ff354040e2cbc676f0320d8c8850d,0x8fb08cb45e1a0032dccd0951812dba7a8ebe5b255bdd...,8,1.455821e+03,2,2018-09-05T21:06:04Z,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,4.222995e+02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16156,16156,2021-03-25T23:19:55Z,22178,0x958f6b67f61b0169a

### Create min and max from bids and merge them to the Master dataframe

In [4]:
df_bids_min = df_bids.groupby('tokenId', as_index=False)['usd'].min()
df_bids_min.rename(columns={'usd': 'usd_min_bids'}, inplace=True)
df_bids = df_bids.merge(df_bids_min, left_on="tokenId", right_on="tokenId")

In [5]:
df_bids_max = df_bids.groupby('tokenId', as_index=False)['usd'].max()
df_bids_max.rename(columns={'usd': 'usd_max_bids'}, inplace=True)
df_bids = df_bids.merge(df_bids_max, left_on="tokenId", right_on="tokenId")

In [6]:
df_bids = df_bids[['tokenId', 'usd_min_bids','usd_max_bids']]
df_bids = df_bids.drop_duplicates(subset = ["tokenId"])

In [7]:
df = df_master.merge(df_bids, left_on="tokenId", right_on="tokenId")

## Create Dummies for words of Kategories

In [8]:
keywords = ['Tweet', 'Twitter', 'Instagram', 'Facebook', 'snapchat', 'posts', 'link', 'socialmedia', 'tag', 'emoji', 'hashtag', 'socail media', 'Portrait', 'Painting', 'contemporary, modern', 'acrylic', 'surrealism', 'minimalism', 'fine art', 'ink', 'painted', 'pencil', 'drawing, expressionism', 'dada', 
                'constructivism', 'futurism', 'cubism', 'impressionism', 'realism', 'Baroque', 'Banksy', 'Graffiti', 'pop art', 'street art', ' old masters', 'abstract', 'Meme', 'gaming', 'comics', 'software', 'gamers', 'mythology', 'gods', 'magic', 'mystic', 'cat', 'dog', 'hero', 'assassin', 'GIF', 'Pixel', '3d', 'animation', 
           'photoshop', 'video', 'image', 'png', 'jpeg', 'mp4', 'iPhone', 'smartphone', 'photography', 'venus', 'Athena', 'TV-series', 'comics', 'phantasy', 'saint', 'galaxy', 'Fire', 'dark', 'dead', 'red', 'murder', 'dystopia', 'broken', 'shadows', 'darkness', 'blood', 'alien', 'violence', 'weapons', 'guns', 'RIP', 'dystopia', 
                'depression', 'anxiety', 'Crypto', 'Blockchain', 'Ethereum', 'Bitcoin', 'ETH', 'digital', 'quantum', 'cryptocurrency', 'digitally', 'cyberspace', 'collage', 'mining', 'NFT', 
          'money', 'tokenized', 'coins', 'token', 'XCopy', 'collage', 'algorithm', 'generate', 'process', 'artificial intelligence', 'machine', 'data science', 'natural science', 'physics', 'computer science', 
      'robot', 'virtual reality', 'data',  'matrix','Nature', 'waterfalls', 'sky', 'beach', 'travel', 'day', 'night', 'island', 'Venice', 'Los Angeles', 'natural', 'sunset', 'snow', 'ocean', 'city', 'season', 'roses', 
          'climate', 'flower', 'adventures', 'rainbow', 'wood']

In [9]:
df['keywords'] = df.namedescriptiontags.str.extract('({0})'.format('|'.join(keywords)), flags=re.IGNORECASE)
df['keywords'] = df['keywords'].str.lower()

In [10]:
df = pd.get_dummies(df, prefix='keywords_', columns=['keywords'])

## Extract most common words

In [11]:
stop_words =  ['the', 'a ','us', 'of', 'in', 'that', 'made', 'we\'ve', 'after','their','his','only','previously', 'by', 'this', 'with', 'a', 'but', 'it', 'or', 'i', 'my', 'be', 'to', 'we', 'you', 'them', 'they', 'at', 'and',
             'is', 'on', 'for', 'an', 'was', 'are', 'have', 'will', 'not', 'can', 'as', 'what', '.', ',', '-', '/', '!', '?', '()', '(', ')', '+', '#', '*', '&', '//', 'has', 'me', 'so', 'our', 'when', 'its', 'all', 'from', 'your', 'into', 'who', 'one', 'which', 'no',
              'more', 'first', 'like', 'through', 'x', 'some', 'these', 'been', 'using']

words_new = (df['namedescriptiontags']
           .str.lower()
           .str.cat(sep=' ')
           .split()
)

common_words = pd.DataFrame(Counter(words_new).most_common(100),
                    columns=['Word', 'Frequency'])

list_words = common_words['Word']
list_words = list_words.to_list()
list_common_words = [i for i in list_words if i not in stop_words]
list_common_words

['art',
 'gif',
 '3d',
 'digital',
 'animation',
 'abstract',
 'cryptoart',
 'illustration',
 'surreal',
 'painting',
 'animated',
 'portrait',
 'loop',
 'created',
 'generative',
 'artwork',
 'digitalart',
 'crypto',
 'series',
 'ai',
 'new',
 '3dart',
 'bitcoin',
 'nft',
 'piece',
 'time',
 'psychedelic',
 'drawing',
 'ethereum',
 'color',
 'glitch',
 'rare',
 'superrare',
 'work',
 'surrealism',
 'original',
 'life',
 'blockchain',
 'world',
 'love',
 'space',
 'gan']

### Create Dummies for most common words

In [12]:
df['common_words'] = df.namedescriptiontags.str.extract('({0})'.format('|'.join(list_common_words)), flags=re.IGNORECASE)
df['common_words'] = df['common_words'].str.lower()

In [13]:
df = pd.get_dummies(df, prefix='common_', columns=['common_words'])

In [14]:
df

,Unnamed: 0,timestamp_x,tokenId,buyer,seller,eth,rate,usd,contract_x,transactionId_x,namedescriptiontags,description_category,image,media,type,size,dimensions,creator,owner,timestamp_y,contract_y,transactionId_y,tokenId_count_bids,usd_mean_bids,tokenId.1,timestamp_x.1,Covid_dummy,keyword_social_media,keyword_art_movement,keyword_gaming,keyword_dark,keyword_crypto,keyword_AI,keyword_nature,keyowrd_other,2018_Q2,2018_Q3,2018_Q4,2019_Q1,2019_Q2,2019_Q3,2019_Q4,2020_Q1,2020_Q2,2020_Q3,2020_Q4,2021_Q1,dollar,usd_min_bids,usd_max_bids,keywords__3d,keywords__abstract,keywords__acrylic,keywords__adventures,keywords__algorithm,keywords__alien,keywords__animation,keywords__anxiety,keywords__artificial intelligence,keywords__assassin,keywords__athena,keywords__banksy,keywords__baroque,keywords__beach,keywords__bitcoin,keywords__blockchain,keywords__blood,keywords__broken,keywords__cat,keywords__city,keywords__climate,keywords__coins,keywords__collage,keywords__comics,keywords__crypto,keywords__cubism,keywords__cyberspace,keywords__dada,keywords__dark,keywords__data,keywords__day,keywords__dead,keywords__depression,keywords__digital,keywords__dog,keywords__dystopia,keywords__emoji,keywords__eth,keywords__ethereum,keywords__facebook,keywords__fine art,keywords__fire,keywords__flower,keywords__futurism,keywords__galaxy,keywords__gamers,keywords__gaming,keywords__generate,keywords__gif,keywords__gods,keywords__graffiti,keywords__guns,keywords__hashtag,keywords__hero,keywords__image,keywords__ink,keywords__instagram,keywords__iphone,keywords__island,keywords__jpeg,keywords__link,keywords__los angeles,keywords__machine,keywords__magic,keywords__matrix,keywords__meme,keywords__minimalism,keywords__mining,keywords__money,keywords__mp4,keywords__murder,keywords__mystic,keywords__mythology,keywords__natural,keywords__nature,keywords__nft,keywords__night,keywords__ocean,keywords__painted,keywords__painting,keywords__pencil,keywords__photography,keywords__photoshop,keywords__physics,keywords__pixel,keywords__png,keywords__pop art,keywords__portrait,keywords__posts,keywords__process,keywords__quantum,keywords__rainbow,keywords__realism,keywords__red,keywords__rip,keywords__robot,keywords__roses,keywords__saint,keywords__season,keywords__shadows,keywords__sky,keywords__smartphone,keywords__snow,keywords__socialmedia,keywords__software,keywords__street art,keywords__sunset,keywords__surrealism,keywords__tag,keywords__token,keywords__tokenized,keywords__travel,keywords__tweet,keywords__twitter,keywords__venice,keywords__venus,keywords__video,keywords__violence,keywords__virtual reality,keywords__waterfalls,keywords__weapons,keywords__wood,keywords__xcopy,common__3d,common__abstract,common__ai,common__animated,common__animation,common__art,common__bitcoin,common__blockchain,common__color,common__created,common__crypto,common__cryptoart,common__digital,common__drawing,common__ethereum,common__gan,common__generative,common__gif,common__glitch,common__illustration,common__life,common__loop,common__love,common__new,common__nft,common__original,common__painting,common__piece,common__portrait,common__psychedelic,common__rare,common__series,common__space,common__superrare,common__surreal,common__time,common__work,common__world
0,0,2018-04-05T23:50:12Z,1,0xbc74c3adc2aa6a85bda3eca5b0e235ca08532772,0x860c4604fe1125ea43f81e613e7afb2aa49546aa,0.46,381.36,1.754256e+02,0x41a322b28d0ff354040e2cbc676f0320d8c8850d,0xf1097e3617632e43b7c0a46ffeb4d741d0a67b25fb06...,"AI Generated Nude Portrait #1 Robbie Barrat, A...",6.0,https://ipfs.pixura.io/ipfs/QmX4ECCmA4HZSPxzsg...,NaN,NaN,NaN,NaN,0x860c4604fe1125ea43f81e613e7afb2aa49546aa,0xd0c0650cd08acd4e9553c48c60c94be04fecce43,2018-04-05T23:20:48Z,0x41a322b28d0ff354040e2cbc676f0320d8c8850d,0xf3e68d3a53b1bb3a2cdb4aa3a6c871626e6dcf7b8df1...,13,1.020731e+04,1,2018-04-05T23:50:12Z,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1.754256e+02,0.51167,110319.000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [15]:
df.to_excel("Master_Data_new2.xlsx")